### Generate examples for DRS Passport revisions

In [25]:
from fasp.search import DiscoverySearchClient, Gen3ManifestClient
from fasp.loc import DRSMetaResolver


from fasp.runner import FASPRunner

faspRunner = FASPRunner(program='GTEX_TCGA_Federated_Analysis.ipynb')
runNote = 'GTEX and TCGA via FASPRunner'

Add comments

In [26]:
import json
# TCGA Query - CRDC
searchClient = DiscoverySearchClient('https://ga4gh-search-adapter-presto-public.prod.dnastack.com/')

queries = ["""
    SELECT  'crdc:'||file_id drs_id
    FROM search_cloud.cshcodeathon.gdc_rel24_filedata_active 
    where data_format = 'BAM' 
    and project_disease_type = 'Breast Invasive Carcinoma'
    limit 3""",
"""
    SELECT  file_id drs_id
    FROM search_cloud.cshcodeathon.gdc_rel24_filedata_active 
    where data_format = 'BAM' 
    and project_disease_type = 'Breast Invasive Carcinoma'
    limit 3"""]


def getSeln(query):
    res = searchClient.runQuery(query)
    seln = []
    for r in res:
        seln.append(r[0])
    #print(json.dumps(res[0:3][0], indent=4))
    selection = {"selection":seln}
    print(json.dumps(selection, indent=4))

for q in queries:
    getSeln(q)


_Retrieving the query_
____Page1_______________
____Page2_______________
____Page3_______________
____Page4_______________
____Page5_______________
____Page6_______________
{
    "selection": [
        "crdc:030e5e74-6461-4f05-a399-de8e470bc056",
        "crdc:04c68898-ddac-4e15-9f9a-5bf278d55e4a",
        "crdc:00589653-5840-4c11-8572-5aa7d00a73f8"
    ]
}
_Retrieving the query_
____Page1_______________
____Page2_______________
____Page3_______________
____Page4_______________
____Page5_______________
{
    "selection": [
        "030e5e74-6461-4f05-a399-de8e470bc056",
        "04c68898-ddac-4e15-9f9a-5bf278d55e4a",
        "00589653-5840-4c11-8572-5aa7d00a73f8"
    ]
}


In [23]:
query = '''select 'crdc:'||drs_id
from search_cloud.cshcodeathon.gecco_gen3_drs_index i
join dbgap_demo.scr_gecco_susceptibility.sample_multi sa on sa.sample_id = i.sample_id
join dbgap_demo.scr_gecco_susceptibility.subject_phenotypes_multi su on su.dbgap_subject_id = sa.dbgap_subject_id
where age between 50 and 55
and affection_status = 'Case'
and file_type = 'cram' limit 3'''
getSeln(query)

_Retrieving the query_
____Page1_______________
____Page2_______________
____Page3_______________
____Page4_______________
____Page5_______________
____Page6_______________
{
    "selection": [
        "crdc:dg.4DFC/89c68e80-0f5b-47de-b01b-8eb789a1061b",
        "crdc:dg.4DFC/fea9649e-1750-43c3-9b87-b314219eb840",
        "crdc:dg.4DFC/8ff3df8b-671e-42e3-a684-0dfa49c20e35"
    ]
}


In [24]:
query = '''select 'sradrs:'||sra_drs_id, sex, age
from search_cloud.cshcodeathon.gecco_sra_drs_index i
join dbgap_demo.scr_gecco_susceptibility.sample_multi sa on sa.sample_id = i.sample_id
join dbgap_demo.scr_gecco_susceptibility.subject_phenotypes_multi su on su.dbgap_subject_id = sa.dbgap_subject_id
where age between 50 and 55
and affection_status = 'Case'
and file_type = 'cram' limit 10'''
getSeln(query)

_Retrieving the query_
____Page1_______________
____Page2_______________
____Page3_______________
____Page4_______________
____Page5_______________
____Page6_______________
____Page7_______________
{
    "selection": [
        "sradrs:4083cc9baac81657edb627635fcfbeb0",
        "sradrs:eb765677453c85f573d744be63a4530e",
        "sradrs:2b070565053062762fed8a1f61fd5c91",
        "sradrs:9324abd1277f50472208dfb6ac15af64",
        "sradrs:3c6840438950a0cdd0fa072dd35854cf",
        "sradrs:4ea5d13b1ede2a4c1578b27853df23d5",
        "sradrs:9e3c1dc17119d6037b14e8269f41f359",
        "sradrs:b4b831b0d4af57754817ef18bc1d4ab3",
        "sradrs:650c2f4f184b040497e0822fe4460874",
        "sradrs:f806c96f31b51747334681a85128eee2"
    ]
}


In [30]:
elixirQuery = "SELECT 'egadrs:'||fileid, filename FROM dbgap_demo.scr_ega.scr_egapancreatic_sample_multi p join dbgap_demo.scr_ega.scr_egapancreatic_files f on f.sample_primary_id = p.sample_primary_id where phenotype = 'pancreatic adenocarcinoma' limit 3"
getSeln(elixirQuery)


_Retrieving the query_
____Page1_______________
____Page2_______________
____Page3_______________
____Page4_______________
____Page5_______________
____Page6_______________
{
    "selection": [
        "egadrs:EGAF00001554155",
        "egadrs:EGAF00001554156",
        "egadrs:EGAF00001554157"
    ]
}


A Search and WES client are then set up to work with the Anvil data

The Search client here  is a placeholder to search a local file. That file contains file ids downloaded as a manifest from the Gen3 Anvil portal. That list of files in that manifest had already been filtered to relevant samples. The anv: DRS prefix was added in an edited version of the file.

#Todo check what access_ids DRSMetaresolver is using for each run

In [4]:
from fasp.workflow import sbcgcWESClient
searchClient = Gen3ManifestClient('../fasp/data/gtex/gtex-cram-manifest_wCuries.json')
# drsClient No need to reset this. DRS Metasolver will pick the right client
#wesClient = sbWESClient(settings['SevenBridgesInstance'], settings['SevenBridgesProject'],
                    #'~/.keys/sbcgc_key.json')

wesClient = sbcgcWESClient(settings['SevenBridgesProject'])

faspRunner.configure(searchClient, drsClient, wesClient)
runList2 = faspRunner.runQuery(3, runNote)


Running query
3
subject=GTEX-1GTWX-0001-SM-7J3A5.cram, drsID=anv:dg.ANV0/76bb893d-12da-41ca-8828-ff89551d3e15
sending id dg.ANV0/76bb893d-12da-41ca-8828-ff89551d3e15 to: anvilDRSClient
workflow submitted, run:37c14324-4e81-413d-9b86-0eca72c9e24d
____________________________________________________________
subject=GTEX-14PQA-0003-SM-7DLH4.cram, drsID=anv:dg.ANV0/66352de8-4b50-4cae-881d-b76d03df5ac8
sending id dg.ANV0/66352de8-4b50-4cae-881d-b76d03df5ac8 to: anvilDRSClient
workflow submitted, run:92d21ed2-53ba-402a-b077-32cb12ac9beb
____________________________________________________________
subject=GTEX-1B98T-0004-SM-7J38T.cram, drsID=anv:dg.ANV0/ed9ac9ae-02da-4e97-93da-ad86aa77d227
sending id dg.ANV0/ed9ac9ae-02da-4e97-93da-ad86aa77d227 to: anvilDRSClient
workflow submitted, run:7dc06985-3e58-4466-9b00-a799b99759f0
____________________________________________________________
